<a href="https://colab.research.google.com/github/tnc-br/ddf_common/blob/gen_isoscape/generate_isoscape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
MODEL_SAVE_LOCATION = "/content/gdrive/MyDrive/amazon_rainforest_files/variational/model/kriging_numerical_stabilized/random_ablated.keras"
TRANSFORMER_SAVE_LOCATION = "/content/gdrive/MyDrive/amazon_rainforest_files/variational/model/kriging_numerical_stabilized/random_ablated_transformer.pkl"
RUN_ID = 'variational_kriging_numerical_stabilized_random_grouped_ablated_2023_08_02'

In [2]:
#@title Imports and modules.

%pip install keras
%pip install tensorflow
%pip install sklearn

from osgeo import gdal, gdal_array
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import List
import pandas as pd
from tqdm import tqdm
import joblib
import os
import sys
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
from sklearn.compose import ColumnTransformer

!if [ ! -d "/content/ddf_common_stub" ] ; then git clone -b test https://github.com/tnc-br/ddf_common_stub.git; fi
sys.path.append("/content/ddf_common_stub/")
import ddfimport
ddfimport.ddf_import_common()

import raster

executing checkout_branch ...
b''
main branch checked out as readonly. You may now use ddf_common imports


# Logic for iterating over brazil landscape and making predictions

In [3]:
def get_predictions_at_each_pixel(
    model: tf.keras.Model,
    feature_transformer: ColumnTransformer,
    geotiffs: dict[str, raster.AmazonGeoTiff],
    bounds: raster.Bounds):

  # Initialize a blank plane representing means and variance.
  predicted_means_np = np.ma.array(
      np.zeros([bounds.raster_size_x, bounds.raster_size_y, 1], dtype=float),
      mask=np.ones([bounds.raster_size_x, bounds.raster_size_y, 1], dtype=bool))
  predicted_vars_np = np.ma.array(
      np.zeros([bounds.raster_size_x, bounds.raster_size_y, 1], dtype=float),
      mask=np.ones([bounds.raster_size_x, bounds.raster_size_y, 1], dtype=bool))

  for x_idx, x in enumerate(tqdm(np.arange(bounds.minx, bounds.maxx, bounds.pixel_size_x, dtype=float))):
    rows = []
    row_indexes = []
    for y_idx, y in enumerate(np.arange(bounds.miny, bounds.maxy, -bounds.pixel_size_y, dtype=float)):
      # Row should contain all the features needed to predict, in the same
      # column order the model was trained.
      row = {}
      row["lat"] = y
      row["long"] = x

      # Surround in try/except as we will be trying to fetch out of bounds data.
      try:
        for geotiff_label, geotiff in geotiffs.items():
          row[geotiff_label] = raster.get_data_at_coords(geotiff, x, y, -1)
          if pd.isnull(row[geotiff_label]):
            raise ValueError
      except (ValueError, IndexError):
        continue # masked and out-of-bounds coordinates

      rows.append(row)
      row_indexes.append((y_idx,0,))

    if (len(rows) > 0):
      X = pd.DataFrame.from_dict(rows)
      if (X.isnull().values.any()):
        print(X.isnull().sum())
      X_scaled = pd.DataFrame(feature_transformer.transform(X),
                              index=X.index, columns=X.columns)
      predictions = model.predict_on_batch(X_scaled)

      means_np = predictions[:, 0]
      for prediction, (y_idx, month_idx) in zip(means_np, row_indexes):
        predicted_means_np.mask[x_idx,y_idx,month_idx] = False # unmask since we have data
        predicted_means_np.data[x_idx,y_idx,month_idx] = prediction
      vars_np = predictions[:, 1]
      for prediction, (y_idx, month_idx) in zip (vars_np, row_indexes):
        predicted_vars_np.mask[x_idx, y_idx, month_idx] = False
        predicted_vars_np.data[x_idx, y_idx, month_idx] = prediction

  return predicted_means_np, predicted_vars_np

# Import rasters

In [4]:
pet_geotiff = raster.load_named_raster(raster.get_raster_path("pet_Stack_mean.tiff"), "pet")
dem_geotiff = raster.load_named_raster(raster.get_raster_path("dem_pa_brasil_raster.tiff"), "dem", use_only_band_index=0)
pa_geotiff = raster.load_named_raster(raster.get_raster_path("dem_pa_brasil_raster.tiff"), "pa", use_only_band_index=1)
krig_means_geotiff = raster.load_named_raster(raster.get_raster_path("uc_davis_d18O_cel_ordinary_random_grouped_means.tiff"), "krig_means")
krig_variances_geotiff = raster.load_named_raster(raster.get_raster_path("uc_davis_d18O_cel_ordinary_random_grouped_vars.tiff"), "krig_variances")

feature_to_geotiff = {
    "VPD" : raster.vapor_pressure_deficit_geotiff(),
    "RH": raster.relative_humidity_geotiff(),
    "PET": pet_geotiff,
    "DEM": dem_geotiff,
    "PA": pa_geotiff,
    "Mean Annual Temperature" : raster.temperature_geotiff(),
    "Mean Annual Precipitation" : raster.brazil_map_geotiff(),
    "ordinary_kriging_linear_d18O_predicted_mean" : krig_means_geotiff,
    "ordinary_kriging_linear_d18O_predicted_variance" : krig_variances_geotiff
}


Driver: GTiff/GeoTIFF
Size is 942 x 936 x 1
Projection is GEOGCS["SIRGAS 2000",DATUM["Sistema_de_Referencia_Geocentrico_para_las_AmericaS_2000",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6674"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4674"]]
Origin = (-74.0, 5.25)
Pixel Size = (0.041666666666666664, -0.041666666666666664)
Driver: GTiff/GeoTIFF
Size is 5418 x 4683 x 2
Projection is GEOGCS["SIRGAS 2000",DATUM["Sistema_de_Referencia_Geocentrico_para_las_AmericaS_2000",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6674"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4674"]]
Origin = (-73.991666667, 5.275)
Pixel Size = (0.008333333333333335, -0.008333333333333333)
Driver: GTiff/GeoTIFF
Size is 5418 x 

# Import Tensorflow model and scalers

In [7]:
model = tf.keras.saving.load_model(MODEL_SAVE_LOCATION)
feature_transformer = joblib.load(TRANSFORMER_SAVE_LOCATION)

In [8]:
def generate_isoscapes_from_variational_model(
    model: tf.keras.Model,
    feature_transformer: ColumnTransformer,
    input_geotiffs: dict[str, raster.AmazonGeoTiff],
    bounds: raster.Bounds):
  means_np, vars_np = get_predictions_at_each_pixel(
    model, feature_transformer, input_geotiffs, bounds)
  raster.save_numpy_to_geotiff(
      bounds,
      means_np,
      raster.get_raster_path(RUN_ID+"_means.tiff"))
  raster.save_numpy_to_geotiff(
      bounds,
      vars_np,
      raster.get_raster_path(RUN_ID+"_vars.tiff"))

bounds = raster.get_extent(dem_geotiff.gdal_dataset)

generate_isoscapes_from_variational_model(
    model, feature_transformer, feature_to_geotiff, bounds)

100%|██████████| 5418/5418 [25:21<00:00,  3.56it/s]


In [4]:
import matplotlib.animation as animation
from matplotlib import rc

rc('animation', html='jshtml')

anim = raster.animate(raster.load_raster(raster.get_raster_path("variational_kriging_numerical_stabilized_random_grouped_ablated_2023_08_02_means.tiff")), 1, 1)

anim

Driver: GTiff/GeoTIFF
Size is 5418 x 4683 x 1
Projection is GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
Origin = (-73.991666667, 5.275)
Pixel Size = (0.008333333333333335, -0.008333333333333333)
..